# Import modules

In [698]:
import numpy as np
import json

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, GRU, Input, TimeDistributed, Activation
from keras.optimizers import RMSprop
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import LambdaCallback


# Get data

In [699]:
with open("../dictionaries/pokemon.json") as f:
    rawData = (json.load(f))
    
# Add escape character '.'
for i in range(len(rawData)):
    rawData[i] += "."
                           
# rawData = np.array(rawData)
# print(rawData.shape)

# One hot encode data

In [700]:
import string
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','.']
def string_vectorizer(strng, nMax):
    vector = [[0 if char != letter else 1 for char in alphabet] 
                  for letter in strng]
    # add padding
    while (len(vector) < nMax):
        vector.append([0 for char in alphabet])
    return vector

In [701]:
nMax = len(max(rawData, key=len))
char_dim = 27
onehot = []
print('max characters is', nMax)

for i in rawData:
    onehot.append(string_vectorizer(i,nMax))
dataClean = np.array(onehot)
print(dataClean.shape)

max characters is 13
(809, 13, 27)


In [702]:
print(dataClean.shape)

(809, 13, 27)


# Train model

In [ ]:
original_dim = (dataClean.shape[1], dataClean.shape[2])
model = Sequential()
# Simple model
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(Dense(char_dim, activation='softmax'))

# model.add(TimeDistributed(Dense(char_dim)))
# model.add(TimeDistributed(Activation('softmax')))


# optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(dataClean,dataClean, batch_size=256, epochs=2000, callbacks=[name_generator])

Epoch 1/2000
809/809 [==============================] - 1s 1ms/step - loss: 2.1404
Names generated after epoch 0:
efwmgufcjika
kqz
hmxomnhscbst
lhjxpjtwkrqa
jnrmhewhyray
Epoch 2/2000
809/809 [==============================] - 0s 323us/step - loss: 2.1258
Epoch 3/2000
809/809 [==============================] - 0s 355us/step - loss: 2.0986
Epoch 4/2000
809/809 [==============================] - 0s 314us/step - loss: 2.0403
Epoch 5/2000
809/809 [==============================] - 0s 350us/step - loss: 1.9657
Epoch 6/2000
809/809 [==============================] - 0s 326us/step - loss: 1.9521
Names generated after epoch 5:
nbk
g
dghxcnnrmra
tdjjbb
jhgwks
Epoch 7/2000
809/809 [==============================] - 0s 326us/step - loss: 1.9206
Epoch 8/2000
809/809 [==============================] - 0s 330us/step - loss: 1.9154
Epoch 9/2000
809/809 [==============================] - 0s 330us/step - loss: 1.8985
Epoch 10/2000
809/809 [==============================] - 0s 328us/step - loss: 1.8914
E

# Generate words

In [14]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 


['/device:CPU:0', '/device:XLA_CPU:0', '/device:GPU:0', '/device:XLA_GPU:0']


In [697]:
def generate_name_loop(epoch, _):
    if epoch % 5 == 0:
        
        print('Names generated after epoch %d:' % epoch)

        for i in range(5):
            print(make_name(model))
        
        
      
name_generator = LambdaCallback(on_epoch_end = generate_name_loop)


def vec_to_string(vec):
    print(vec.shape)
    maxList = vec.argmax(axis=1)
    print(maxList)

    
def make_name(model):
    alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','.']
    name = []
    x = np.zeros((1, nMax, char_dim))
    end = False
    i = 0
    while end==False and i < nMax-1:
        probs = list(model.predict(x)[0,i])
        
        probs = probs / np.sum(probs)
        index = np.random.choice(range(char_dim), p=probs)
        if index == char_dim -1:
            end = True
            break
        else:
            name.append(alphabet[index])
        i += 1

    return ''.join(name)

name = []
while len(name) < 6:
    name = make_name(model)

print(name)

egtgee
